In [68]:
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import LabelEncoder
import pickle
import os 

In [69]:
df = pd.read_csv('/Users/sanatwalia/Desktop/Assignments_applications/Revenue_Optimization-/dataset/mobile_game_iap_dataset.csv')

In [70]:
df.describe()

,Level Reached,Prior Purchases,Price (₹)
count,5000.000000,5000.00000,5000.000000
mean,20.560600,4.49600,335.923600
std,11.517884,4.04065,248.254397
min,1.000000,0.00000,49.000000
25%,11.000000,1.00000,145.000000
50%,21.000000,3.00000,235.000000
75%,31.000000,6.00000,495.000000
max,40.000000,15.00000,999.000000


In [71]:
df.head()

,Date of Purchase,Date of Install,User Segment,Item-Purchased,Session Time,Level Reached,Prior Purchases,Purchase Type,Promo Applied,Price (₹)
0,04/02/2023,30/01/2023,Casual,Season Pass,01h 40m,7,1,Subscription,No,424
1,19/01/2023,29/12/2022,Hardcore,Premium Weapon,02h 33m,11,8,One-time,No,244
2,24/01/2023,05/01/2023,Casual,Limited-Time Offer Pack,01h 54m,35,2,One-time,No,90
3,21/01/2023,28/12/2022,Casual,Ultra Bundle,02h 58m,16,4,One-time,No,655
4,09/02/2023,08/02/2023,Whales,Power-Up Bundle,00h 39m,4,7,One-time,No,167


In [72]:
df.tail()

,Date of Purchase,Date of Install,User Segment,Item-Purchased,Session Time,Level Reached,Prior Purchases,Purchase Type,Promo Applied,Price (₹)
4995,29/01/2023,27/01/2023,Casual,Coin Pack - 1000,02h 06m,35,4,One-time,No,94
4996,18/01/2023,21/12/2022,Hardcore,Premium Weapon,02h 39m,10,10,One-time,No,199
4997,18/01/2023,05/01/2023,Casual,Character Skin,01h 47m,9,2,One-time,No,118
4998,14/02/2023,05/02/2023,Casual,Coin Pack - 1000,02h 38m,5,3,One-time,No,95
4999,11/02/2023,10/02/2023,Casual,Limited-Time Offer Pack,01h 49m,32,3,One-time,Yes,84


In [73]:
columns =df.columns

In [74]:
# finding the unique values in the columns
for col in columns:
    print(f"Unique values in {col}: {df[col].nunique()}")
    # print(f"Unique values in {col}: {df[col].unique()}")
    print("\n")

Unique values in Date of Purchase: 31


Unique values in Date of Install: 60


Unique values in User Segment: 3


Unique values in Item-Purchased: 10


Unique values in Session Time: 176


Unique values in Level Reached: 40


Unique values in Prior Purchases: 16


Unique values in Purchase Type: 2


Unique values in Promo Applied: 2


Unique values in Price (₹): 765




In [75]:
# from nuniqeue values we can see that  purchase type , promo applied and items purchased are categorical variables
# checking for null values
df.isnull().sum()

Date of Purchase    0
Date of Install     0
User Segment        0
Item-Purchased      0
Session Time        0
Level Reached       0
Prior Purchases     0
Purchase Type       0
Promo Applied       0
Price (₹)           0
dtype: int64

In [76]:
# checking for duplicates 
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 0


In [77]:
#checking for the data types of the columns 
data_types = df.dtypes    
print("Data types of the columns:")
print(data_types)


Data types of the columns:
Date of Purchase    object
Date of Install     object
User Segment        object
Item-Purchased      object
Session Time        object
Level Reached        int64
Prior Purchases      int64
Purchase Type       object
Promo Applied       object
Price (₹)            int64
dtype: object


In [78]:
# converting session time  objects in total session time in minutes 
def convert_session_time_to_minutes(session_str):
    """
    Converts session time from format 'Xh Ym' to total minutes as an integer.
    Example: '02h 39m' -> 159
    """
    try:
        parts = session_str.strip().lower().replace(" ", "")
        hours = int(parts.split('h')[0])
        minutes = int(parts.split('h')[1].replace('m', ''))
        return hours * 60 + minutes
    except:
        return None  # or np.nan if you're using pandas


In [79]:
df["Session Time"] = df["Session Time"].apply(convert_session_time_to_minutes)


In [80]:
df.head()

,Date of Purchase,Date of Install,User Segment,Item-Purchased,Session Time,Level Reached,Prior Purchases,Purchase Type,Promo Applied,Price (₹)
0,04/02/2023,30/01/2023,Casual,Season Pass,100,7,1,Subscription,No,424
1,19/01/2023,29/12/2022,Hardcore,Premium Weapon,153,11,8,One-time,No,244
2,24/01/2023,05/01/2023,Casual,Limited-Time Offer Pack,114,35,2,One-time,No,90
3,21/01/2023,28/12/2022,Casual,Ultra Bundle,178,16,4,One-time,No,655
4,09/02/2023,08/02/2023,Whales,Power-Up Bundle,39,4,7,One-time,No,167


In [81]:
# converting object of date of purchase and install 
df["Date of Purchase"] = pd.to_datetime(df["Date of Purchase"], format="%d/%m/%Y")
df["Date of Install"] = pd.to_datetime(df["Date of Install"], format="%d/%m/%Y")

# Create new feature 
df["Days Installed Before Last Purchase"] = (df["Date of Purchase"] - df["Date of Install"]).dt.days


In [82]:
df.head()

,Date of Purchase,Date of Install,User Segment,Item-Purchased,Session Time,Level Reached,Prior Purchases,Purchase Type,Promo Applied,Price (₹),Days Installed Before Last Purchase
0,2023-02-04,2023-01-30,Casual,Season Pass,100,7,1,Subscription,No,424,5
1,2023-01-19,2022-12-29,Hardcore,Premium Weapon,153,11,8,One-time,No,244,21
2,2023-01-24,2023-01-05,Casual,Limited-Time Offer Pack,114,35,2,One-time,No,90,19
3,2023-01-21,2022-12-28,Casual,Ultra Bundle,178,16,4,One-time,No,655,24
4,2023-02-09,2023-02-08,Whales,Power-Up Bundle,39,4,7,One-time,No,167,1


In [83]:
print(df['User Segment'].nunique())
print(df['Purchase Type'].nunique() )
print(df['Promo Applied'].nunique())
print(df['Item-Purchased'].nunique())



3
2
2
10


In [84]:
# usin label encoder to convert categorical variables into numerical values
# Initialize LabelEncoder
label_encoder = LabelEncoder()

In [85]:
columns_to_encode = ['User Segment', 'Purchase Type', 'Promo Applied', 'Item-Purchased']
for col in columns_to_encode:
    df[col] = label_encoder.fit_transform(df[col])



In [86]:
# Ensure the directory exists
save_dir = '/Users/sanatwalia/Desktop/Assignments_applications/Revenue_Optimization-/models'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Save the label encoder as a .pkl file in the 'saved_models' directory
pickle_file_path = os.path.join(save_dir, 'label_encoder.pkl')

with open(pickle_file_path, 'wb') as f:
    pickle.dump(label_encoder, f)

print(f'Label encoder saved to {pickle_file_path}')

Label encoder saved to /Users/sanatwalia/Desktop/Assignments_applications/Revenue_Optimization-/models/label_encoder.pkl


In [87]:
df.head()

,Date of Purchase,Date of Install,User Segment,Item-Purchased,Session Time,Level Reached,Prior Purchases,Purchase Type,Promo Applied,Price (₹),Days Installed Before Last Purchase
0,2023-02-04,2023-01-30,0,8,100,7,1,1,0,424,5
1,2023-01-19,2022-12-29,1,7,153,11,8,0,0,244,21
2,2023-01-24,2023-01-05,0,5,114,35,2,0,0,90,19
3,2023-01-21,2022-12-28,0,9,178,16,4,0,0,655,24
4,2023-02-09,2023-02-08,2,6,39,4,7,0,0,167,1


In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Date of Purchase                     5000 non-null   datetime64[ns]
 1   Date of Install                      5000 non-null   datetime64[ns]
 2   User Segment                         5000 non-null   int64         
 3   Item-Purchased                       5000 non-null   int64         
 4   Session Time                         5000 non-null   int64         
 5   Level Reached                        5000 non-null   int64         
 6   Prior Purchases                      5000 non-null   int64         
 7   Purchase Type                        5000 non-null   int64         
 8   Promo Applied                        5000 non-null   int64         
 9   Price (₹)                            5000 non-null   int64         
 10  Days Install

In [90]:
df.describe()

,Date of Purchase,Date of Install,User Segment,Item-Purchased,Session Time,Level Reached,Prior Purchases,Purchase Type,Promo Applied,Price (₹),Days Installed Before Last Purchase
count,5000,5000,5000.00000,5000.000000,5000.000000,5000.000000,5000.00000,5000.00000,5000.000000,5000.000000,5000.000000
mean,2023-01-30 02:49:55.200000,2023-01-14 18:49:49.440000,0.49540,4.504400,94.250800,20.560600,4.49600,0.19500,0.306600,335.923600,15.333400
min,2023-01-15 00:00:00,2022-12-16 00:00:00,0.00000,0.000000,5.000000,1.000000,0.00000,0.00000,0.000000,49.000000,1.000000
25%,2023-01-22 00:00:00,2023-01-06 00:00:00,0.00000,2.000000,50.000000,11.000000,1.00000,0.00000,0.000000,145.000000,8.000000
50%,2023-01-30 00:00:00,2023-01-15 00:00:00,0.00000,5.000000,95.500000,21.000000,3.00000,0.00000,0.000000,235.000000,15.000000
75%,2023-02-07 00:00:00,2023-01-24 00:00:00,1.00000,7.000000,139.000000,31.000000,6.00000,0.00000,1.000000,495.000000,23.000000
max,2023-02-14 00:00:00,2023-02-13 00:00:00,2.00000,9.000000,180.000000,40.000000,15.00000,1.00000,1.000000,999.000000,30.000000
std,NaN,NaN,0.66908,2.890891,50.816855,11.517884,4.04065,0.39624,0.461128,248.254397,8.655243


In [91]:
df.to_csv('/Users/sanatwalia/Desktop/Assignments_applications/Revenue_Optimization-/dataset/mobile_game_iap_dataset_cleaned.csv', index=False)
print("Cleaned dataset saved as mobile_game_iap_dataset_cleaned.csv")

Cleaned dataset saved as mobile_game_iap_dataset_cleaned.csv
